<a href="https://colab.research.google.com/github/mattwantshouses/REAPI_App/blob/main/Combine_output_files_into_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combine the output files into one

Use this when you have files in one google drive folder that need to be combined.

Designed when the columns are all the same.


---

Instructions:

1.   In Google drive, copy the folder ID that has the files
     Ex: in https://drive.google.com/drive/folders/1dq-zCFoHpl30_f7BRquB_q5wOJnQsxWQ the folder ID = `1dq-zCFoHpl30_f7BRquB_q5wOJnQsxWQ`
2.   Run the cells in order.
3.   When prompted choose the files from the menu.
4.   Make sure it completes then go check the drive folder for the file.



In [ ]:
import pandas as pd
import io
import sys
import traceback
from google.colab import drive, auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from datetime import datetime

# Authenticate and create the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3', cache_discovery=False)

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Helper Functions

def print_status(message):
    print(f"Status: {message}")
    sys.stdout.flush()

# File Operations in Google Drive
# List files in the folder
def list_files_in_folder(folder_id):
    
    print_status("Listing files in the folder...")
    try:
        results = drive_service.files().list(
            q=f"'{folder_id}' in parents and (mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' or mimeType='application/vnd.ms-excel' or mimeType='text/csv') and trashed=false",
            fields="files(id, name, mimeType)"
        ).execute()
        return results.get('files', [])
    except Exception as e:
        print(f"Error listing files: {str(e)}")
        traceback.print_exc()
        return []

# Download a file from Google Drive given its ID
def download_file(file_id):
    print_status(f"Downloading file {file_id}...")
    try:
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}% complete")
        fh.seek(0)
        return fh
    except Exception as e:
        print(f"Error downloading file: {str(e)}")
        traceback.print_exc()
        return None

# Upload a file to Google Drive in a specified folder
def upload_file(name, content, mime_type, folder_id):

    print_status(f"Uploading file {name}...")
    try:
        file_metadata = {
            'name': name,
            'parents': [folder_id]
        }
        media = MediaFileUpload(content, mimetype=mime_type, resumable=True)
        file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        return file.get('id')
    except Exception as e:
        print(f"Error uploading file: {str(e)}")
        traceback.print_exc()
        return None

# Main Function for Combining Files

def combine_files(folder_id):
    # List files in the folder
    files = list_files_in_folder(folder_id)

    if not files:
        print("No spreadsheet files found in the specified folder.")
        return

    # Display available files and handle file selection
    print("\nAvailable files:")
    for i, file in enumerate(files):
        print(f"{i+1}. {file['name']}")

    while True:
        print("\nFile selection options:")
        print("- Enter 'all' to select all files")
        print("- Enter file numbers separated by commas (e.g., '1,3,5' to select files 1, 3, and 5)")
        print("- Enter a range of files (e.g., '1-5' to select files 1 through 5)")
        selection = input("Your selection: ")

        if selection.lower() == 'all':
            selected_indices = range(len(files))
            break
        elif ',' in selection:
            try:
                selected_indices = [int(idx.strip()) - 1 for idx in selection.split(',')]
                if all(0 <= idx < len(files) for idx in selected_indices):
                    break
                else:
                    print("Invalid selection. Please ensure all numbers are within the range of available files.")
            except ValueError:
                print("Invalid input. Please enter numbers separated by commas.")
        elif '-' in selection:
            try:
                start, end = map(int, selection.split('-'))
                if 1 <= start <= end <= len(files):
                    selected_indices = range(start-1, end)
                    break
                else:
                    print("Invalid range. Please ensure the range is within the available files.")
            except ValueError:
                print("Invalid input. Please enter a valid range (e.g., '1-5').")
        else:
            print("Invalid input. Please try again.")

    print("\nSelected files:")
    for idx in selected_indices:
        print(f"- {files[idx]['name']}")

    # Process selected files
    df_list = []
    total_files = len(selected_indices)
    for i, idx in enumerate(selected_indices):
        file = files[idx]
        file_id = file['id']
        file_name = file['name']
        mime_type = file['mimeType']

        print_status(f"Processing file {i+1} of {total_files}: {file_name}")
        file_content = download_file(file_id)

        if file_content is None:
            print(f"Skipping file {file_name} due to download error.")
            continue

        try:
            if mime_type == 'text/csv':
                df = pd.read_csv(file_content)
            else:  # Excel files
                df = pd.read_excel(file_content)
            df_list.append(df)
        except Exception as e:
            print(f"Error processing file {file_name}: {str(e)}")
            traceback.print_exc()

        print(f"Progress: {((i+1) / total_files) * 100:.2f}% complete")

    if not df_list:
        print("No files were successfully processed. Exiting.")
        return

    # Combine dataframes and prepare output file
    print_status("Combining dataframes...")
    combined_df = pd.concat(df_list, ignore_index=True)

    # Generate output file name
    first_file_name = files[selected_indices[0]]['name'][:20]
    current_date = datetime.now().strftime("%m-%d-%y")
    output_filename = f"combined {first_file_name} {current_date}.csv"

    # Prepare and save the output file
    print_status("Preparing output file...")
    output_content = combined_df.to_csv(index=False)

    print_status("Saving output file...")
    try:
        with open(output_filename, 'wb') as f:
            f.write(output_content.encode('utf-8'))
    except Exception as e:
        print(f"Error saving output file: {str(e)}")
        traceback.print_exc()
        return

    # Upload the combined file to Google Drive
    print_status("Uploading combined file to Google Drive...")
    file_id = upload_file(output_filename, output_filename, 'text/csv', folder_id)

    if file_id:
        print(f"Combined spreadsheet '{output_filename}' has been saved to the same Google Drive folder.")
        print(f"File ID: {file_id}")
    else:
        print("Failed to upload the combined file to Google Drive.")

# Main execution
try:
    folder_id = input("Enter the Google Drive folder ID: ")
    combine_files(folder_id)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")
    traceback.print_exc()

Enter the Google Drive folder ID: 1dq-zCFoHpl30_f7BRquB_q5wOJnQsxWL
Status: Listing files in the folder...

Available files:
1. test 07-16-24.csv
2. FL_preforeclosure_batch_10_converted.csv
3. FL_preforeclosure_batch_9_converted.csv
4. FL_preforeclosure_batch_5_converted.csv
5. FL_preforeclosure_batch_8_converted.csv
6. .txt_converted.csv
7. FL_preforeclosure_batch_7_converted.csv
8. FL_preforeclosure_batch_2_converted.csv
9. FL_preforeclosure_batch_3_converted.csv
10. FL_preforeclosure_batch_4_converted.csv
11. FL_preforeclosure_batch_1_converted.csv
12. FL_preforeclosure_batch_6_converted.csv
13. propgpt results - FL land parcels, pre-foreclosure or tax deliquent.csv
14. FL_size_100.csv
15. propgpt results - orange park preforeclosures.csv
16. propgpt results - jacksonville private money lenders 12 months.csv

File selection options:
- Enter 'all' to select all files
- Enter file numbers separated by commas (e.g., '1,3,5' to select files 1, 3, and 5)
- Enter a range of files (e.g., '